## Overview of Utilities

The utils module provides two main utility functions:

1. **`format_ai_message()`** - Converts structured AI responses to LangChain message objects
2. **`get_tool_descriptions()`** - Extracts and parses tool descriptions from Python functions for LLM consumption

## Setup

In [3]:
from utils.utils import format_ai_message, get_tool_descriptions
import json

# Part 1: `format_ai_message()` Function

## Purpose
Converts a structured response object (containing answer and tool_calls) into a LangChain `AIMessage` object. This is essential for maintaining message history in agentic workflows and ensuring OpenAI compatibility.

## Internal Logic

The function creates an OpenAI-compatible message flow:

1. **Checks if response contains `tool_calls`**
   - If tool_calls exist: Creates AIMessage with both `content` and `tool_calls`
   - If no tool_calls: Creates simple AIMessage with just `content` (answer)

2. **Formats tool_calls for OpenAI compatibility:**
   - Assigns unique `id` to each tool call (format: `call_{index}`)
   - Extracts `name` and `arguments` from each tool call
   - Creates structured dict: `{"id": "call_0", "name": "...", "arguments": {...}}`
   - This format is OpenAI's standard for function calling

3. **Creates AIMessage with proper structure:**
   - Returns `AIMessage(content=answer, tool_calls=[...])`
   - This message becomes part of the conversation history
   - Can be converted back to OpenAI message format via `convert_to_openai_messages()`

## Hands-on Example: format_ai_message()

In [4]:
# Define a mock response class to demonstrate format_ai_message
class MockResponse:
    def __init__(self, answer, tool_calls=None):
        self.answer = answer
        self.tool_calls = tool_calls or []

# Example 1: Response without tool calls
response_no_tools = MockResponse(
    answer="The weather is sunny today."
)

message1 = format_ai_message(response_no_tools)
print("Example 1: Response without tool calls")
print(f"Message type: {type(message1)}")
print(f"Content: {message1.content}")
print(f"Tool calls: {message1.tool_calls}")
print()

Example 1: Response without tool calls
Message type: <class 'langchain_core.messages.ai.AIMessage'>
Content: The weather is sunny today.
Tool calls: []



In [5]:
# Example 2: Response with tool calls
class MockToolCall:
    def __init__(self, name, arguments):
        self.name = name
        self.arguments = arguments

tool_calls = [
    MockToolCall(name="get_weather", arguments={"location": "New York"}),
    MockToolCall(name="get_temperature", arguments={"location": "New York"})
]

response_with_tools = MockResponse(
    answer="I'll check the weather for you.",
    tool_calls=tool_calls
)

message2 = format_ai_message(response_with_tools)
print("Example 2: Response with tool calls")
print(f"Message type: {type(message2)}")
print(f"Content: {message2.content}")
print(f"Number of tool calls: {len(message2.tool_calls)}")
print("\nTool calls details:")
for tc in message2.tool_calls:
    print(f"  - ID: {tc['id']}, Name: {tc['name']}, Args: {tc['args']}")

Example 2: Response with tool calls
Message type: <class 'langchain_core.messages.ai.AIMessage'>
Content: I'll check the weather for you.
Number of tool calls: 2

Tool calls details:
  - ID: call_0, Name: get_weather, Args: {'location': 'New York'}
  - ID: call_1, Name: get_temperature, Args: {'location': 'New York'}


# Part 2: `get_tool_descriptions()` Function

## Purpose
Extracts detailed tool descriptions from Python functions and converts them into a structured format suitable for LLMs to understand available tools and their usage.

## Internal Logic

The function uses Python's AST (Abstract Syntax Tree) to:

1. **Extract function metadata:**
   - Function name
   - Docstring (description)
   - Type hints for parameters and return values

2. **Parse parameter information:**
   - Parameter names
   - Parameter types (from type hints)
   - Parameter descriptions (from docstring)
   - Default values (if any)
   - Required parameters (those without defaults)

3. **Convert to OpenAI schema format:**
   - Converts Python types to JSON schema types (str→string, int→integer, etc.)
   - Creates a structured JSON schema compatible with OpenAI function calling

## Hands-on Example: get_tool_descriptions()

In [6]:
# Define sample functions with proper docstrings and type hints
def calculate_total_price(quantity: int, unit_price: float, tax_rate: float = 0.1) -> float:
    """Calculate total price including tax.
    
    Args:
        quantity: Number of items to purchase.
        unit_price: Price per item in dollars.
        tax_rate: Tax rate as a decimal (default 0.1 for 10%).
    
    Returns:
        The total price including tax.
    """
    subtotal = quantity * unit_price
    return subtotal * (1 + tax_rate)


def search_products(query: str, category: str = "all", limit: int = 10) -> list:
    """Search for products in the catalog.
    
    Args:
        query: The search term.
        category: Product category filter.
        limit: Maximum number of results to return.
    
    Returns:
        List of matching products with their details.
    """
    # Mock implementation
    return []


# Get descriptions for these functions
tools = [calculate_total_price, search_products]
tool_descriptions = get_tool_descriptions(tools)

print("Extracted Tool Descriptions:")
print(json.dumps(tool_descriptions, indent=2))

Extracted Tool Descriptions:
[
  {
    "name": "calculate_total_price",
    "description": "Calculate total price including tax.",
    "parameters": {
      "type": "object",
      "properties": {
        "quantity": {
          "type": "integer",
          "description": "Number of items to purchase."
        },
        "unit_price": {
          "type": "number",
          "description": "Price per item in dollars."
        },
        "tax_rate": {
          "type": "number",
          "description": "Tax rate as a decimal (default 0.1 for 10%).",
          "default": 0.1
        }
      }
    },
    "required": [
      "quantity",
      "unit_price"
    ],
    "returns": {
      "type": "number",
      "description": "The total price including tax."
    }
  },
  {
    "name": "search_products",
    "description": "Search for products in the catalog.",
    "parameters": {
      "type": "object",
      "properties": {
        "query": {
          "type": "string",
          "descriptio

In [ ]:
# Let's examine the structure of a single tool description
print("\n" + "="*60)
print("Detailed breakdown of calculate_total_price tool:")
print("="*60)

tool_desc = tool_descriptions[0]

print(f"\n1. Name: {tool_desc['name']}")
print(f"\n2. Description: {tool_desc['description']}")
print(f"\n3. Parameters:")
for param_name, param_info in tool_desc['parameters']['properties'].items():
    print(f"   - {param_name}:")
    print(f"     • Type: {param_info['type']}")
    print(f"     • Description: {param_info.get('description', 'N/A')}")
    if 'default' in param_info:
        print(f"     • Default: {param_info['default']}")

print(f"\n4. Required Parameters: {tool_desc['required']}")
print(f"\n5. Return Type: {tool_desc['returns']['type']}")
print(f"\n6. Return Description: {tool_desc['returns']['description']}")

In [ ]:
# Practical use case: Using tool descriptions for LLM prompts
print("\nHow LLMs see the tools (JSON Schema format):")
print("=" * 60)

# This is exactly what gets sent to the LLM
for tool in tool_descriptions:
    print(f"\nTool: {tool['name']}")
    print(f"Description: {tool['description']}")
    print(f"\nSchema:")
    print(json.dumps({
        "type": "object",
        "properties": tool['parameters']['properties'],
        "required": tool['required']
    }, indent=2))

## Key Features of Tool Description Extraction

| Feature | Benefit |
|---------|----------|
| Automatic docstring parsing | No manual schema definition needed |
| Type hint conversion | JSON schema compatible with LLMs |
| Default value detection | LLMs can understand optional parameters |
| Parameter descriptions | Clear guidance for LLMs on parameter usage |
| Return type info | LLMs know what to expect from tool output |

## Real-World Integration Example

Here's how these utilities work together in an agentic workflow:

In [7]:
# Simulated agent workflow
print("Agentic Workflow with Utilities:")
print("="*60)

print("\n1. SETUP PHASE:")
print("   - Define functions with docstrings and type hints")
print("   - Call get_tool_descriptions() to extract schemas")
print(f"   - Available tools: {[t['name'] for t in tool_descriptions]}")

print("\n2. LLM EXECUTION PHASE:")
print("   - Send tool descriptions to LLM")
print("   - LLM decides which tool to call and with what arguments")

print("\n3. RESPONSE FORMATTING PHASE:")
print("   - LLM returns structured response with tool_calls")
print("   - Call format_ai_message() to convert to AIMessage")
print("   - Add message to conversation history")

# Simulate the response
mock_llm_response = MockResponse(
    answer="I'll calculate the total price for you.",
    tool_calls=[MockToolCall(
        name="calculate_total_price",
        arguments={"quantity": 5, "unit_price": 29.99, "tax_rate": 0.08}
    )]
)

formatted_message = format_ai_message(mock_llm_response)
print(f"\n   Result: AIMessage with {len(formatted_message.tool_calls)} tool call(s)")
print(f"   Ready to execute: {formatted_message.tool_calls[0]['name']}()")

Agentic Workflow with Utilities:

1. SETUP PHASE:
   - Define functions with docstrings and type hints
   - Call get_tool_descriptions() to extract schemas
   - Available tools: ['calculate_total_price', 'search_products']

2. LLM EXECUTION PHASE:
   - Send tool descriptions to LLM
   - LLM decides which tool to call and with what arguments

3. RESPONSE FORMATTING PHASE:
   - LLM returns structured response with tool_calls
   - Call format_ai_message() to convert to AIMessage
   - Add message to conversation history

   Result: AIMessage with 1 tool call(s)
   Ready to execute: calculate_total_price()


## Summary

### `format_ai_message()`
- **When to use**: After receiving LLM response with tool calls
- **Input**: Response object with `answer` and optional `tool_calls`
- **Output**: LangChain `AIMessage` ready for message history
- **Key benefit**: Standardizes message format across the workflow

### `get_tool_descriptions()`
- **When to use**: During agent setup to define available tools
- **Input**: List of Python functions
- **Output**: JSON-compatible tool schemas for LLMs
- **Key benefit**: Eliminates manual schema writing via introspection

These utilities enable **code-first tool definitions** where the function itself is the source of truth!